# Before starting

For the first point, We started downloading the yellow cab data for January 2018(yellow_tripdata_2018-01.csv). Our plan was to make our studies and analysis firstly for one month and then extend our results considering also other years.
Our Data-set contains a really large amount of data so we opted to choose the simple pd.read_csv() module from pandas.
After considering many options such as Dask DataFrames, we decided to use the pandas importing method together with the chucksize option. The solution to working with a massive file with eight milions of lines is to load the file in smaller chunks and analyze with the smaller chunks. Here is our code: 

In [1]:
import pandas as pd
def data_aggregator(path,columnnumber,chunksize):
    df_list = []
    for chunk in pd.read_csv(path,usecols=columnnumber, chunksize=chunksize):
        df_list.append(pd.DataFrame(chunk))
    result = pd.concat(df_list)
    del df_list
    result1=result.fillna(value=0)
    del result
    return result1
JanData="yellow_tripdata_2018-01.csv" #We define the path of our data(of January)
JanDF=data_aggregator(JanData,[1,2,4,7,16],10000)
print(JanDF.head())
#With this function we can easily import data we need. We select the columns and the dataset we desire and, as we said, 
#we use chunksize the make the function run faster(and to avoid memory problems). We have printed the head of the dataframe to
#show that everything was correct and the df contained columns corresponding to [1,2,4,7,16] indexs.

  tpep_pickup_datetime tpep_dropoff_datetime  trip_distance  PULocationID  \
0  2018-01-01 00:21:05   2018-01-01 00:24:23            0.5            41   
1  2018-01-01 00:44:55   2018-01-01 01:03:05            2.7           239   
2  2018-01-01 00:08:26   2018-01-01 00:14:21            0.8           262   
3  2018-01-01 00:20:22   2018-01-01 00:52:51           10.2           140   
4  2018-01-01 00:09:18   2018-01-01 00:27:06            2.5           246   

   total_amount  
0          5.80  
1         15.30  
2          8.30  
3         34.80  
4         16.55  


After defining the function to import the data correctly, We were asked to merge our data with a .json file in the homework repo  ([.json](https://github.com/CriMenghini/ADM-2018/blob/master/Homework_2/taxi_zone_lookup.csv)). The data was embedd in the page source-code so we had to apply our knowledge in web-scraping ang get the lines of sourcecode we needed. We used the **Beautiful** package. We firstly collected all the location ids, then the boroughs, then the zones and finally the serving zones. We have done that noticing that in the sourcecode all the above information required where all starting with a <td> and we simply went trough all the values starting with it. After that we merged all together in a Pandas DataFrame called boroghFrame.

In [2]:
import requests
from bs4 import BeautifulSoup
page = requests.get("https://github.com/CriMenghini/ADM-2018/blob/master/Homework_2/taxi_zone_lookup.csv")
soup = BeautifulSoup(page.content, 'html.parser')
ids=[]
bor=[]
zon=[]
srv_zon=[]
cell=0
for i in range(2,1327,5):    #Firstly We get all the location ids
    a=soup.find_all('td')[i].get_text()  
    ids.append(a)
    cell=cell+1
    
cell=0
for i in range(3,1328,5):    #Then all the boroughs
    a=soup.find_all('td')[i].get_text()
    bor.append(a)
    cell=cell+1
    
cell=0
for i in range(4,1329,5):   #After that we get al zones
    a=soup.find_all('td')[i].get_text()
    zon.append(a)
    cell=cell+1
cell=0
for i in range(5,1330,5):    #Finally we get all the seving zones
    a=soup.find_all('td')[i].get_text()
    srv_zon.append(a)
    cell=cell+1
# after getting all the informations needed we merged all the lists together
data_tuples = list(zip(ids,bor,zon,srv_zon))
boroughFrame=pd.DataFrame(data_tuples,columns = ["PULocationID", "Borogh", "Zone", "srv_zon"]) #name columns assigned
boroughFrame['PULocationID']=boroughFrame['PULocationID'].apply(int) #We wanted to be sure that PuLocationId were all ints.




Then, we finally merged our Dataframes defining this function. We merged using the PULocation id column. **So we assumed that the LocationID in the .json file was equal to PULocationId in taxidataset.**

In [3]:
def data_aggregator2(df1,df2,oncolumns,jointype):
    result = pd.merge(df1, boroughFrame, on=oncolumns,how=jointype)
    return result
JanData="yellow_tripdata_2018-01.csv" #We define the path of our data(of January)
JanBoroughData=data_aggregator2(JanDF,boroughFrame,['PULocationID'],"inner")  
print(len(JanBoroughData))
del JanDF,boroughFrame,page,soup,ids,bor,zon,srv_zon 
#we also clean our memory deleting all the valuables we do not need. 

8759874


We also noticed, exploring our dataset, that we had some values simply not reasonable and we decided to get rid of them. For example, for some rows the trip distance was equal to 0 or negative and that values would have impacted our analysis. We lost approximately 55000 rows

In [5]:
JanBoroughData = JanBoroughData[JanBoroughData.trip_distance >0]
print(len(JanBoroughData))

8704498
